In [ ]:
'''
Done: getting ndcg-value for debateorg.json 
Running: getting debateorg corpus re-created by deepct
'''

In [1]:
!pip install python-terrier
!pip install json
import pyterrier as pt
if not pt.started():
  pt.init()

     |████████████████████████████████| 95 kB 4.4 MB/s 
     |████████████████████████████████| 1.1 MB 49.4 MB/s 
     |████████████████████████████████| 69 kB 9.6 MB/s 
     |████████████████████████████████| 222 kB 57.0 MB/s 
     |████████████████████████████████| 596 kB 58.4 MB/s 
     |████████████████████████████████| 6.3 MB 29.8 MB/s 
     |████████████████████████████████| 1.8 MB 35.4 MB/s 
     |████████████████████████████████| 294 kB 72.2 MB/s 
     |████████████████████████████████| 126 kB 68.8 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 291 kB 66.7 MB/s 
     |████████████████████████████████| 45 kB 3.7 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.1-py3-none-any.whl size=102452 sha256=c557f196f787ab967e057fe450448e53b7ce72e2c1e8cc999f122434b79c1317
  Stored in directory: /root/.cache/pip/wheels/c0/9a/c8/1c2d9ec6a1494bb54f47e0d2627b5ed7b2de704b66723d3417
  Created wheel for ir-measures: filen

In [2]:
from typing import List, Set, Callable

def tokenize_text(text: str, stopwords: Set[str] = None) -> List[str]:
    if stopwords is None:
        stopwords = set()

    raw_tokens = re.split('\W', text)
    normalized_tokens = map(lambda w: w.lower().strip(".:,;?!"), raw_tokens)
    return ' '.join(list(filter(lambda w: w != '' and w not in stopwords, normalized_tokens)))

In [3]:
import ijson
import re
data_set = set()
id_text = {}
# only parliamentary.json from args.me corpus
# filename = "/content/drive/MyDrive/workspace/parliamentary.json"
filename = "/content/drive/MyDrive/workspace/debateorg.json"
with open(filename, 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'arguments.item')
    for i, o in enumerate(objects):
        id_text[i] = str(i), o['id'], tokenize_text(o["premises"][0]["text"])

In [4]:
import pandas as pd
test_df = pd.DataFrame.from_dict(id_text).transpose()
test_df = test_df.rename(columns={0: "docid", 1: "docno", 2: "text"})

In [5]:
pd_indexer = pt.DFIndexer("./pd_index")
indexref = pd_indexer.index(test_df["text"], test_df["docno"])

09:17:38.301 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (Sb38112c8-A443a9828) - further warnings are suppressed
09:22:15.776 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1686 empty documents
09:22:16.544 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key S2db48a61-A430a7cb1 is not unique: 265281,131561
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)


In [6]:
from xml.etree import ElementTree
tree = ElementTree.parse('/content/drive/MyDrive/workspace/topics.xml')
data = tree.getroot()
titles = []
title_ids = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    title_ids.append(num)


In [7]:
topics = pd.DataFrame(titles,columns = ['query'])
topics['qid'] = topics.index + 51
columns_titles = ["qid","query"]
topics=topics.reindex(columns=columns_titles)

In [8]:
index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)

In [9]:
qrels_path=("/content/drive/MyDrive/workspace/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

In [10]:
eval = pt.Utils.evaluate(res,qrels,metrics=["map", "ndcg"])
eval

{'map': 0.32764634921508384, 'ndcg': 0.5676840565415017}

In [11]:
import os
if not os.path.exists("uncased_L-12_H-768_A-12.zip"):
  !wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
  !unzip uncased_L-12_H-768_A-12.zip
  !mkdir -p bert-base-uncased
  !mv vocab.txt bert_* bert-base-uncased/

--2021-11-17 09:22:34--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.33.208, 172.217.9.208, 172.217.12.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.33.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 408102251 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 389.20M  38.6MB/s    in 7.4s    

2021-11-17 09:22:41 (52.3 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [408102251/408102251]

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: bert_model.ckpt.data-00000-of-00001  
  inflating: bert_config.json        
  inflating: vocab.txt               
  inflating: bert_model.ckpt.index   


In [12]:
%tensorflow_version 1.x

!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_deepct.git

TensorFlow 1.x selected.
  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-0qr9nmaf/python-terrier_18c92245ff73438f99b21281ae81ef27
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-0qr9nmaf/python-terrier_18c92245ff73438f99b21281ae81ef27
  Cloning https://github.com/terrierteam/pyterrier_deepct.git to /tmp/pip-req-build-oohov4b8
  Running command git clone -q https://github.com/terrierteam/pyterrier_deepct.git /tmp/pip-req-build-oohov4b8
  Cloning https://github.com/cmacdonald/DeepCT.git (to revision tf1) to /tmp/pip-install-d25p22bb/deepct_f74210c0b1ff4a9da0181a1a8f2f1951
  Running command git clone -q https://github.com/cmacdonald/DeepCT.git /tmp/pip-install-d25p22bb/deepct_f74210c0b1ff4a9da0181a1a8f2f1951
  Running command git checkout -b tf1 --track origin/tf1
  Switched to a new branch 'tf1'
  Branch 'tf1' set up to track remote branch 'tf1' from 'origin'.
  Created wheel for pyterrier-deepct: filename=pyterrier_

In [13]:
if not os.path.exists("marco.zip"):
  !wget http://boston.lti.cs.cmu.edu/appendices/arXiv2019-DeepCT-Zhuyun-Dai/outputs/marco.zip
  !unzip marco.zip

--2021-11-17 09:23:14--  http://boston.lti.cs.cmu.edu/appendices/arXiv2019-DeepCT-Zhuyun-Dai/outputs/marco.zip
Resolving boston.lti.cs.cmu.edu (boston.lti.cs.cmu.edu)... 128.2.207.139, 128.2.207.32
Connecting to boston.lti.cs.cmu.edu (boston.lti.cs.cmu.edu)|128.2.207.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1237248594 (1.2G) [application/zip]
Saving to: ‘marco.zip’

marco.zip           100%[===================>]   1.15G  36.8MB/s    in 30s     

2021-11-17 09:23:44 (39.8 MB/s) - ‘marco.zip’ saved [1237248594/1237248594]

Archive:  marco.zip
   creating: marco/
  inflating: marco/checkpoint        
  inflating: marco/graph.pbtxt       
  inflating: marco/model.ckpt-65816.data-00000-of-00001  
  inflating: marco/model.ckpt-65816.index  
  inflating: marco/model.ckpt-65816.meta  
  inflating: marco/train.tf_record   


In [14]:
import pyterrier_deepct
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
assert tf.__version__.startswith("1"), "TF 1 is required by DeepCT; on Colab, use %tensorflow_version 1.x"
tf.logging.set_verbosity(tf.logging.ERROR)
deepct = pyterrier_deepct.DeepCTTransformer("bert-base-uncased/bert_config.json", "marco/model.ckpt-65816")

In [15]:
super_df = test_df.copy()
# super_df

In [16]:
# !rm -rf /content/pd_super_index

In [18]:
xd = test_df.head().copy()
xd

,docid,docno,text
0,0,Sb38112c8-A443a9828,done
1,1,Sb38112c8-A48f87c39,let s get this over with
2,2,Sb38112c8-A85a910e8,you have not stated a resolution position you ...
3,3,Sb38112c8-A96d2d2c7,should schools have open campus why or why not...
4,4,S4468ab0b-Af7993e79,it seems we are at an impasse 1 in rounds 1 3 ...


In [19]:
deepct.transform(xd)

,docno,text
0,Sb38112c8-A443a9828,done done done done done done done done done d...
1,Sb38112c8-A48f87c39,let let let let s s s s s get get get get get ...
2,Sb38112c8-A85a910e8,have not not stated stated stated stated state...
3,Sb38112c8-A96d2d2c7,schools schools schools schools schools school...
4,S4468ab0b-Af7993e79,at impasse impasse impasse impasse impasse imp...


In [ ]:
super_df = deepct.transform(super_df)

In [ ]:
super_df.to_csv(index=False, path_or_buf='/content/drive/MyDrive/workspace/superdf.csv')

In [ ]:
super_df

In [ ]:
pd_super_indexer = pt.DFIndexer("./pd_super_index")
index_super_ref = pd_super_indexer.index(super_df["text"], super_df["docno"])

In [ ]:
index_super = pt.IndexFactory.of(index_super_ref)

In [ ]:
retr_super = pt.BatchRetrieve(index_super, controls = {"wmodel": "BM25"})
res_super=retr_super.transform(topics)

In [ ]:
res_super

In [ ]:
# # dataset = pt.get_dataset("irds:vaswani")
# # index_loc = "./deepct_index_path"
# indexer = deepct >> pd_indexer
# indexref = pd_indexer.index(test_df["text"], test_df["docno"])
